# 影像旋轉

In [2]:
import os
import re
import math
import random
import cv2 as cv
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [3]:
# true_box_PATH = '../test_img/txt/20220808_anno_test_lines_0.txt'       # 這邊要修改train/test 的路徑!!!!!!!!
true_box_PATH = '../dataset/txt/20220822_anno_train_lines_0.txt'
# true_box_PATH = '../dataset/txt/20220817_anno_val_lines_0.txt'

true_bboxes = {}
with open(true_box_PATH) as f:
    for line in f.readlines():
        true_bboxes[line.split(' ')[0]] = tuple([int(i) for i in line.split(' ')[1].split(',')[:4]])


print('訓練集box 數量= ', len(true_bboxes))

訓練集box 數量=  134


# 旋轉影像

In [11]:
class RandomRotate(object):
    def __init__(self, max_degree, fix_angle, rotate_ratio=0.5, mean=(0,0,0)):   # mean=(103.939, 116.779, 123.68)
        assert isinstance(max_degree, int)
        self.max_degree = max_degree
        self.fix_angle  = fix_angle
        self.ratio      = rotate_ratio
        self.mean       = mean

    def bbox_rotate(self, bbox, M, img_shape):
        """Flip bboxes horizontally.
        Args:
            bbox(list): [left, right, up, down]
            img_shape(tuple): (height, width)
        """
        assert len(bbox) == 4
        a = M[:, :2]  ##a.shape (2,2)
        b = M[:, 2:]  ###b.shape(2,1)
        b = np.reshape(b, newshape=(1, 2))
        a = np.transpose(a)

        [left, right, up, down] = bbox
        corner_point = np.array([[left, up], [right, up], [left, down], [right, down]])
        corner_point = np.dot(corner_point, a) + b
        min_left     = max(int(np.min(corner_point[:, 0])), 0)             # max
        max_right    = min(int(np.max(corner_point[:, 0])), img_shape[1])  # min
        min_up       = max(int(np.min(corner_point[:, 1])), 0)             # max
        max_down     = min(int(np.max(corner_point[:, 1])), img_shape[0])  # min

        return [min_left, max_right, min_up, max_down]

    def __call__(self, image, bbox=None):
        height, width, _ = image.shape
        if random.random() < self.ratio:
            angle = random.uniform(-self.max_degree, self.max_degree)
            # fix_angle = self.fix_angle    # 改旋轉角度!!!!!!!!!!!!!!!!!
            rotate_matrix = cv.getRotationMatrix2D((width / 2, height / 2), angle, 1.0)
            image         = cv.warpAffine(image, rotate_matrix, (width, height), flags=cv.INTER_LINEAR,
                                           borderMode=cv.BORDER_CONSTANT, borderValue=self.mean)

            if bbox:
                bbox = self.bbox_rotate(bbox, rotate_matrix, (height, width))
        return image, bbox


In [ ]:
def RotateImage(img, img_name, true_bbox,fix_angle=0):
    [left, up, right, down] = true_bbox
    bbox = [left, right, up, down]
    ro   = RandomRotate(max_degree=180, fix_angle=fix_angle, rotate_ratio=1, mean=(0, 0, 0))  # mean:背景

    new_img, new_bbox = ro(img, bbox=bbox)

    # plot
    # cv.rectangle(img, (bbox[0], bbox[2]), (bbox[1], bbox[3]), (0, 255, 0), 2)
    # cv.rectangle(new_img, (new_bbox[0], new_bbox[2]), (new_bbox[1], new_bbox[3]), (0, 0, 255), 2)
    # plt.subplot(121)
    # plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    # plt.subplot(122)
    # plt.imshow(cv.cvtColor(new_img, cv.COLOR_BGR2RGB))
    # plt.show()
   
    new_bbox = [new_bbox[0],new_bbox[2],new_bbox[1],new_bbox[3]]

    return new_img, new_bbox

In [9]:
def bbox_2_anno_txt_file(TXT_PATH, img_name, bbox):
    ## output file
    list_file = open(TXT_PATH, 'a')
    
    ## Write object information to .txt file
    list_file.write(img_name)
    cls_id = 0   # 壺腹乳頭類別:0
    list_file.write(" " + ",".join([str(a) for a in bbox]) + ',' + str(cls_id))
    list_file.write('\n')
    list_file.close()

# 固定角度

In [ ]:
# IMG_PATH       = '../dataset/20220808_train_vali_img_sopbox_0/'       # 這邊要修改train 的路徑!!!!!!!!
# train_TXT_PATH = '../dataset/txt/20220808_anno_train_lines_0.txt'     # !! txt要先創
# val_TXT_PATH   = '../dataset/txt/20220808_anno_val_lines_0.txt'       # !! txt要先創

# train = ['a', 'b', 'c', 'e', 'f','g', 'j']
# val   = ['d', 'i']
# test  = ['h', 'z']

# fix_angles = [90, 180, 270]   # 固定角度

# for img_name in true_bboxes:
#         original_img = cv.imread(IMG_PATH + img_name)
#         true_bbox    = true_bboxes[img_name]                       
#         for fix_angle in fix_angles:
#             new_img, new_bbox = RotateImage(img=original_img, img_name=img_name, fix_angle=fix_angle, true_bbox=true_bbox)
#             new_img_name      = '{}_'.format(str(fix_angle)) + img_name            
            
#             # 影像寫進去
#             cv.imwrite(IMG_PATH+new_img_name , new_img)
#             # bbox寫進去
#             bbox_2_anno_txt_file(TXT_PATH=val_TXT_PATH,  # 這邊要修改train/val 的路徑!!!!!!!!
#                                  img_name=new_img_name, 
#                                  bbox=new_bbox) 
#             # print(new_bbox)


In [4]:
IMG_PATH       = '../dataset/20220822_train_vali_img_sopbox_0/'       
train_TXT_PATH = '../dataset/txt/20220822_anno_train_lines_0.txt'     # !! txt要先創
val_TXT_PATH   = '../dataset/txt/20220822_anno_val_lines_0.txt'       # !! txt要先創

train = ['a', 'b', 'c', 'e', 'f','g', 'j']
val   = ['d', 'i']
test  = ['h', 'z']

# 隨機角度 

In [ ]:
for img_name in true_bboxes:
    original_img = cv.imread(IMG_PATH + img_name)
    true_bbox    = true_bboxes[img_name]                       
    for i in range(1,4):    # 隨機產生3組角度
        new_img, new_bbox = RotateImage(img=original_img, img_name=img_name, true_bbox=true_bbox)
        new_img_name      = 'angle_{}_'.format(i) + img_name            
        
        # 影像寫進去
        cv.imwrite(IMG_PATH+new_img_name , new_img)
        # bbox寫進去
        bbox_2_anno_txt_file(TXT_PATH=val_TXT_PATH,  # 這邊要修改train/val 的路徑!!!!!!!!
                                img_name=new_img_name, 
                                bbox=new_bbox) 
        # print(new_bbox)

# 鏡像

In [5]:
def flip_bbox(bbox, size, y_flip=False, x_flip=False):
    """Flip bounding boxes accordingly.
    Args:
        bbox (~numpy.ndarray): See the table below.
        size (tuple): A tuple of length 2. The height and the width
            of the image before resized.
        y_flip (bool): Flip bounding box according to a vertical flip of
            an image.
        x_flip (bool): Flip bounding box according to a horizontal flip of
            an image.
    .. csv-table::
        :header: name, shape, dtype, format
        :obj:`bbox`, ":math:`(R, 4)`", :obj:`float32`, 
        ":math:`(y_{min}, x_{min}, y_{max}, x_{max})`"
    Returns:
        ~numpy.ndarray:
        Bounding boxes flipped according to the given flips.
    """
    H, W = size
    bbox = bbox.copy()
    if y_flip:
        y_max = H - bbox[:, 0]
        y_min = H - bbox[:, 2]
        bbox[:, 0] = y_min
        bbox[:, 2] = y_max
    if x_flip:
        x_max = W - bbox[:, 1]
        x_min = W - bbox[:, 3]
        bbox[:, 1] = x_min
        bbox[:, 3] = x_max
    return bbox

In [10]:
for img_name in true_bboxes:
    print(img_name[:-4])
    original_img = cv.imread(IMG_PATH + img_name)
    true_bbox    = true_bboxes[img_name]

    H,W,c = original_img.shape
    size  = (H,W)

    true_bbox     = np.array(list(true_bbox)).reshape((1,4))      # 整理成numpy array [[x1,y1,x2,y2]]
    new_true_bbox = flip_bbox(bbox=true_bbox, size=size, y_flip=False, x_flip=True)
    new_true_bbox = new_true_bbox[0]    # 鏡像的bbox

    new_img = cv.flip(original_img, 0)  # 1:鏡像y軸 0:鏡像x軸

    # 影像寫進去
    new_img_name = "FlipX_" + img_name
    cv.imwrite(IMG_PATH + new_img_name  , new_img)
    # bbox寫進去
    bbox_2_anno_txt_file(TXT_PATH=train_TXT_PATH, img_name=new_img_name, bbox=new_true_bbox)    # TXT_PATH要修改!!! 
    


a_000001
a_000002
a_000003
a_000004
a_000005
a_000006
a_000007
a_000008
a_000009
a_000010
b_000051
b_000052
b_000053
b_000054
b_000055
b_000056
b_000057
b_000058
c_000035
c_000036
c_000037
c_000038
c_000039
c_000040
c_000041
c_000042
c_000043
c_000044
c_000045
c_000046
c_000047
c_000048
c_000049
c_000050
c_000051
e_000026
e_000027
e_000028
e_000029
e_000030
e_000031
e_000032
e_000033
e_000034
e_000035
e_000036
e_000037
e_000038
e_000039
e_000040
e_000041
e_000042
e_000043
e_000044
e_000045
e_000046
e_000047
e_000048
e_000049
e_000050
e_000051
e_000052
e_000053
e_000054
f_000016
f_000017
f_000018
f_000020
f_000021
f_000022
f_000023
f_000024
f_000025
f_000026
f_000027
f_000028
f_000029
f_000030
f_000031
f_000032
f_000033
f_000034
f_000035
f_000036
f_000037
f_000038
f_000039
f_000040
f_000041
f_000042
f_000043
f_000044
f_000045
f_000046
f_000047
f_000048
f_000049
f_000050
f_000051
f_000052
f_000053
g_000026
g_000027
g_000030
g_000031
g_000032
g_000033
g_000034
g_000035
g_000037
g_000039
g

: 

In [ ]:
# plt.imshow(cv.cvtColor(img_array, cv.COLOR_BGR2RGB)) 
# plt.show()

# # plot

# original_img_array_box = cv.rectangle(original_img_array, (bff[0], bff[1]), (bff[2], bff[3]), (0, 255, 0), 2)
# x1,y1,x2,y2 = bff[0],bff[1],bff[2],bff[3]
# x = [x1,x2]
# y = [y1,y2]
# plt.scatter(x,y,color='red',)
# plt.imshow(cv.cvtColor(original_img_array_box, cv.COLOR_BGR2RGB))
# plt.show()

# print("box= ", bff)

# 銳化

In [ ]:
def sharpen(img, sigma=100):    
    # sigma = 5、15、25
    blur_img = cv.GaussianBlur(img, (0, 0), sigma)
    usm      = cv.addWeighted(img, 1.5, blur_img, -0.5, 0)

    return usm


In [ ]:
for img_name in true_bboxes:
    original_img = cv.imread(IMG_PATH + img_name)
    # 銳化
    sharpen_img = sharpen(original_img)
    # 影像寫進去
    cv.imwrite(IMG_PATH + img_name  , sharpen_img)
    print(img_name, "finish")
    
    plt.subplot(121)
    plt.imshow(cv.cvtColor(original_img, cv.COLOR_BGR2RGB))
    plt.subplot(122)
    plt.imshow(cv.cvtColor(sharpen_img, cv.COLOR_BGR2RGB))
    plt.show()

# RandomErasing

In [21]:
from random_eraser import get_random_eraser
eraser = get_random_eraser(p=1, s_l=0.02, s_h=0.1, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=True)


for img_name in true_bboxes:
    original_img = cv.imread(IMG_PATH + img_name)
    for i in range(1,6):
        img_erase = eraser(original_img.copy())
        new_img_name = "erase_{}_".format(i) + img_name
        # # 影像寫進去
        # cv.imwrite(IMG_PATH + new_img_name  , img_erase)

        # bbox寫進去
        true_bbox = true_bboxes[img_name]
        bbox_2_anno_txt_file(TXT_PATH=train_TXT_PATH, img_name=new_img_name, bbox=true_bbox)

left=  184 top=  26
left=  724 top=  237
left=  508 top=  538
left=  418 top=  400
left=  116 top=  311
left=  313 top=  669
left=  418 top=  663
left=  394 top=  556
left=  588 top=  72
left=  238 top=  376
left=  54 top=  527
left=  354 top=  296
left=  567 top=  66
left=  131 top=  547
left=  371 top=  552
left=  156 top=  429
left=  259 top=  454
left=  67 top=  705
left=  177 top=  546
left=  441 top=  420
left=  212 top=  270
left=  644 top=  599
left=  325 top=  735
left=  369 top=  668
left=  254 top=  300
left=  51 top=  384
left=  55 top=  467
left=  421 top=  303
left=  284 top=  487
left=  348 top=  230
left=  217 top=  542
left=  521 top=  25
left=  77 top=  360
left=  683 top=  94
left=  101 top=  355
left=  228 top=  518
left=  705 top=  340
left=  669 top=  673
left=  699 top=  205
left=  176 top=  562
left=  458 top=  53
left=  734 top=  104
left=  115 top=  25
left=  169 top=  194
left=  17 top=  199
left=  162 top=  636
left=  757 top=  383
left=  744 top=  503
left=